SECOND ASSIGNMENT OF ENERJISA & PATIKA BOOTCAMP
* [OneHotEncoding modelin görmediği veriye nasıl uygulanır?](#1)
* [Labelencoding'de ilgili kolon için ölçeklendirme nasıl yapılır?](#2)
* [Imbalance datasette train&test ayırıken neleri göz önünde bulundurmalıyız?](#3)
* [Validation dataseti](#4)
* [Predict_proba & treshold](#5)

In [22]:
import pandas as pd
import numpy as np

<a name="1"></a>
#### 1. One Hot Encoding

##### OneHotEncoding Nedir?
One Hot Encoding, 
kategorik değişkenlerin ikili (binary) olarak temsil edilmesi anlamına gelmektedir.

Bu işlem ilk önce kategorik değerlerin tamsayı değerleriyle eşlenmesini gerektirir. 

Daha sonra, her bir tamsayı değeri, 1 ile işaretlenmiş tamsayı indeksi dışında ki tüm değerleri   sıfır  olan bir ikili vektör olarak temsil edilir. 

In [23]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

In [24]:
temp = [['Male', 1], ['Female', 3], ['Female', 2]]
encoder.fit(temp)
encoder.categories_

[array(['Female', 'Male'], dtype=object), array([1, 2, 3], dtype=object)]

In [25]:
encoder.transform([['Female', 1], ['Male', 4]]).toarray()

array([[1., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.]])

In [26]:
encoder.inverse_transform([[0, 1, 1, 0, 0], [0, 0, 0, 1, 0]])

array([['Male', 1],
       [None, 2]], dtype=object)

****
<a name="2"></a>
#### 2. LabelEncoding'de ilgili kolon için ölçeklendirme nasıl yapılır? 

##### LabelEncoding Nedir?
Veriyi birebir sayısallaştırmaya yarar. Yani kategorik her veriye sayısal bir değer atar.

In [31]:
bridge_types = ('Arch','Beam','Truss','Cantilever','Tied Arch','Suspension','Cable')
bridge_df = pd.DataFrame(bridge_types, columns=['Bridge_Types'])

In [32]:
bridge_df['Bridge_Types'] = bridge_df['Bridge_Types'].astype('category')

In [33]:
bridge_df['Bridge_Types_Cat'] = bridge_df['Bridge_Types'].cat.codes
bridge_df

,Bridge_Types,Bridge_Types_Cat
0,Arch,0
1,Beam,1
2,Truss,6
3,Cantilever,3
4,Tied Arch,5
5,Suspension,4
6,Cable,2


With Sci-kit Learn Library

In [35]:
from sklearn.preprocessing import LabelEncoder

# creating initial dataframe
bridge_types1 = ('Arch','Beam','Truss','Cantilever','Tied Arch','Suspension','Cable')
bridge_df1 = pd.DataFrame(bridge_types1, columns=['Bridge_Types'])
# creating instance of labelencoder
labelencoder = LabelEncoder()
# Assigning numerical values and storing in another column
bridge_df1['Bridge_Types_Cat'] = labelencoder.fit_transform(bridge_df1['Bridge_Types'])
bridge_df1

,Bridge_Types,Bridge_Types_Cat
0,Arch,0
1,Beam,1
2,Truss,6
3,Cantilever,3
4,Tied Arch,5
5,Suspension,4
6,Cable,2


****
<a name="3"></a>
#### 3. Imbalance datasette train test split yaparken neleri göz önünde bulundurmalıyız? 

```
Dengesiz Veri Kümesi Nedir?
* Sınıflandırma yaparken, sınıfların eşit dağılmadığı, yani her sınıf için yaklaşık olarak aynı sayıda verinin olmadığı veri kümesidir. 
* Mesela, ikili sınıflandırma (binary classification) durumunda, 500 verinin olduğu bir veri kümesindeki;
40 verinin azınlık sınıfa, 460 verinin ise diğer sınıfa ait olması durumudur.
```

```
* Dengesiz veri kümeleriyle karşılaştığımız durumlarda birçok makine öğrenmesi algoritması, 
sınıflardaki eşit olmayan dağılımı dikkate almayarak güvenilir sonuçlar vermeyebiliyor. 
* Yani elde edilen sınıflandırıcılar (classifiers), 
genel hata oranını minimize etmeye çalışırken
ağırlıklı olan sınıfı (Sınıf-0) kategorize ederek, azınlık sınıfını (Sınıf-1) göz ardı edebiliyor
```

```
1. DOĞRU METRİK SEÇİMİ
Öncelikle, birçok sınıflandırmada kullanılan ve eğitilen algoritmaların
performansını ölçmek için kullanılan doğruluk (accuracy), dengesiz veri 
kümesiyle çalışılan durumlarda, model performansını ölçmek için güvenilir bir 
metrik değildir.
* Duyarlılık (Precision)
* Hassasiyet (Recall)
* F1 ölçümü
* ROC Eğrisi (ROC Curve)
* Duyarlılık-Hassasiyet Eğrisi (Precision-Recall Curve)
* Hata Matrisi (Confusion Matrix) metriklerini baz almak daha doğru olacaktır.
```

```
2)KATMANLAMA (STRATIFICATION)
Dengesiz veri kümelerini, 
eğitim veri kümesi (training dataset) ve test veri kümesi (test dataset) 
olarak ikiye ayırırken, verilerin dengesizlik oranında, 
eğitim ve test kümelerine eşit dağılması gerekir.
Yani her iki kümede sınıfların oranının aynı olması, modelin performansının 
istikrarlı (consistency) olması açısından önemli.

****

Dolayısıyla, veri kümesini eğitim ve test veri kümelerine ayırırken; 
train_test_split() fonksiyonundaki “stratify” parametresiyle; 
çapraz doğrulama skorunu (cross-validation score) hesaplarken; 
hiper parametreleri (hyper-parameters) 
GridSearchCV veya RandomizedSearchCV 
ile belirlerken katmanlama durumu dikkate almamız gerekir.
```

```
3) YENİDEN ÖRNEKLEME (RESAMPLING)
Yeniden örnekleme yaparak, 
dengesiz veri kümelerini daha dengeli hale getirebiliriz. 
Bunu yapmak için ilk yöntem azınlık sınıfına ait verileri
çeşitli yöntemlerle arttırarak eşit sayıda veriye sahip sınıflar elde etmek. 
(oversampling)

****

* Naive Random Over-Sampling azınlık sınıfa ait var olan veriyi rastgele seçip 
yerine koymak (replacement) suretiyle kopyalayarak dengeyi sağlamak.


* MOTE (Synthetic Minority Oversampling Technique) ve 
ADASYN (Adaptive Synthetic Sampling Method) algoritmalarında 
interpolasyon yöntemiyle sentetik veriler üreterek denge sağlanabilir.


* Undersampling ise ağırlıklı sınıfa ait verileri
veri kümesinden çıkararak dengeli bir veri kümesi elde etmek.

* Random undersampling, ClusterCentroids ve NearMiss teknikleri veri kümesini 
dengeye getirmek için kullanılabilir. Eğer çalıştığınız veri kümesi yeterince 
büyük değilse bu yöntem, bilgi kaybına sebep olacağından diğer yöntemler tercih 
edilmelidir.
```

```
4) SINIF AĞIRLIKLARINI AYARLAMAK (ADJUSTING CLASS WEIGHTS)
Eğer elinizde dengesiz bir veri kümesi varsa, 
“class_weight” parametresi yardımıyla 
azınlık sınıfına atanan ağırlığı dengesizlik oranında arttırarak, 
algoritmanın azınlık verisini yanlış sınıflandırmasından kaynaklanan hata oranını arttırabilirsiniz. 
Dolayısıyla, modeli oluştururken genel hata oranını (overall error rate) azaltmaya çalışan algoritma bu dengesizliği, 
yani azınlık sınıfı dikkate alacak ve performansı artacaktır.
```

```
5) PENALIZED MODELLER
Penalized Sınıflandırma yöntemleri 
(penalized-SVM and penalized-LDA, logistik regresyon vs.) modele, 
algoritmaları eğitirken, yanlış sınıflandırılan azınlık sınıfı verileri için fazladan cost yüklediği için, 
modelin azınlık sınıfına verdiği ağırlığı arttırıyor.
```

```
6) ANOMALİ TESPİTİ (ANOMALY DETECTION)
Anomali tespiti, az rastlanır durumların tespiti demektir. Dengesiz veri 
kümelerinin olduğu durumlarda azınlık sınıfı verilerini, anomali sınıfına 
aitmiş gibi düşünerek, anomali tespiti yaparak sınıfları ayırabiliriz.

*DBSCAN (Density-based Spatial Clustering)
*Kümeleme tabanlı algoritmalar (Clustering-Based Anomaly Detection)
*Isolation Forest Anomali Tespit Algoritması
*Local Outlier Factor Algoritması
*Destekçi Vektör Makinesi Anomali Tespit Algoritması (Support Vector Machine Anomaly Detection Algorithm)
```

****
<a name="4"></a>
#### 4. Validation dataseti (modelin görmediği) nasıl oluşturulur ve nasıl predict etmeye hazır hale getirilir?

```
Validation bölümü train veri seti içinden seçilir. 
Train veri seti üzerinde doğru model seçimi yapılarak algoritma belirlenir. Validation bölümünde ise uygulanan model iyileştirilmeye çalışılır. 
Bunun için hiper parametrik (hyperparameter tuning) uygulamalar denenerek en optimum katsayılar/ağırlıklar bulunmaya çalışılır. 
Özellikle çok büyük veri setleri üzerinde sürekli train datası üzerinden 
çalışılamayacağı için küçük bir bölüm alınarak validation olarak tanımlanır.
```

****
<a name="5"></a>
#### 4.  Predict_proba metoduyla oran nasıl hesaplanır ve treshold nasıl değiştirilir?

Sınıflandırma problemlerinde gözlemlerin sınıflara ait olma olasılıkları elde etmek istendğinde predict_proba fonksiyonunu kullanmak gerekir. Örneğin bir karar ağacı oluşturuldu ve oluşturulan karar ağacı için olasılıklar öğrenilmek istendiğinde predict_proba metodu kullanılır. Sonuçlar her etikete ait olma yüzdesini içerir.

```
y_pred_proba = dt.predict_proba(X_test)
```

```
probabilities = dt.predict_proba(X_test)

threshold = 0.4
good = probabilities[:, 1]
predicted_good = good > threshold
```

****